In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import keras
import imghdr
import os
import cv2
import warnings
import PIL.Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\aditi\AppData\Local\Temp\ipykernel_18172\1722930265.py:5: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
data = tf.keras.utils.image_dataset_from_directory("melanoma_cancer_dataset/train")

Found 9605 files belonging to 2 classes.


In [4]:
train_size = int(0.8 * len(data))
val_size =len(data) - train_size

In [5]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test =  tf.keras.utils.image_dataset_from_directory("melanoma_cancer_dataset/test")

Found 1000 files belonging to 2 classes.


In [6]:
def preprocess_image(image, label):
    # Normalize the image to the range [0, 1]
    image = image / 255.0
    return image, label

# Apply the preprocessing function to the dataset
train = train.map(preprocess_image)
test = test.map(preprocess_image)
val = val.map(preprocess_image)

In [7]:
logdir = "logs"
tensorflow_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

: 

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define your model
model = Sequential([
    Conv2D(16, (3, 3), activation="relu", input_shape=(256, 256, 3)),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(5, activation="sigmoid")  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Update for multi-class classification
              metrics=['accuracy'])


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit the model
history = model.fit(
    train,
    epochs=20,
    validation_data=val,
    callbacks=[early_stopping]
)

C:\Users\aditi\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


In [ ]:
fig = plt.figure()
plt.plot(history.history["loss"],color="teal",label="loss")
plt.plot(history.history["val_loss"],color="Orange",label="val_loss")
fig.suptitle("Loss",fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history["accuracy"],color="teal",label="accuracy")
plt.plot(history.history["val_accuracy"],color="Orange",label="val_accuracy")
fig.suptitle("Accuracy",fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.models import save_model
model.save('cnn_model_updated.h5')